In [8]:
import xarray
import xarray_beam as xbeam

source_dataset, source_chunks = xbeam.open_zarr('gcs://gcp-public-data-arco-era5/ar/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2')

In [10]:
import pandas as pd
start_date = pd.to_datetime('2016-12-31')
end_date = pd.to_datetime('2021-12-31')
source_dataset = source_dataset.sel(time=slice(start_date, end_date))

In [16]:
print(source_dataset.latitude.size)

721


In [1]:
import sys,os
import torch

sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(os.path.abspath(os.path.dirname(os.getcwd()))))))
from datasets.weather_bench import WeatherDataset
from models.VariableEncoder.datasets.dataset import CustomDataset


air = ["geopotential", 'specific_humidity', 'temperature', 'u_component_of_wind', 'v_component_of_wind', 'vertical_velocity']
sur = ['2m_temperature', '10m_u_component_of_wind', '10m_v_component_of_wind', 'mean_sea_level_pressure', 'total_cloud_cover', 'total_precipitation']
only_input_variable = ['toa_incident_solar_radiation']
constant_variable = ['geopotential_at_surface', 'land_sea_mask']

device = ("cuda" if torch.cuda.is_available() else "cpu" )
device = torch.device(device)

weather = WeatherDataset(0, device=device, offline=True)
src_var_list = weather.get_var_code(air, sur + only_input_variable+constant_variable)
tgt_var_list = weather.get_var_code(air, sur)

데이터셋 불러오는 중...


In [3]:
print(tgt_var_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [11]:
def denormalize(inputs, mean_std):
    # min_max 텐서를 적절히 재구성하여 inputs의 차원에 맞춤
    mean = mean_std[:, 0].view(1, 1, mean_std.size(0), 1)
    std = mean_std[:, 1].view(1, 1, mean_std.size(0), 1)
    # 역정규화 수행
    denormalized = (inputs * std) + mean
    return denormalized

In [15]:
src, tgt, label = dataset[0]

reversed_predict = denormalize(tgt.unsqueeze(0), mean_std)

In [22]:
print(reversed_predict[0, : , :, :label.size(-1)])

tensor([[[2.0130e+05, 2.0128e+05, 2.0127e+05,  ..., 1.9970e+05,
          1.9969e+05, 1.9969e+05],
         [1.6036e+05, 1.6036e+05, 1.6036e+05,  ..., 1.5738e+05,
          1.5740e+05, 1.5741e+05],
         [1.3633e+05, 1.3634e+05, 1.3634e+05,  ..., 1.3227e+05,
          1.3227e+05, 1.3226e+05],
         ...,
         [1.0308e+05, 1.0309e+05, 1.0310e+05,  ..., 1.0268e+05,
          1.0268e+05, 1.0269e+05],
         [1.9595e-01, 1.4695e-01, 1.1039e-01,  ..., 7.8946e-01,
          8.8265e-01, 9.0048e-01],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.8327e-04,
          2.0125e-04, 1.4473e-04]],

        [[2.0138e+05, 2.0138e+05, 2.0137e+05,  ..., 1.9979e+05,
          1.9979e+05, 1.9978e+05],
         [1.6041e+05, 1.6041e+05, 1.6041e+05,  ..., 1.5743e+05,
          1.5745e+05, 1.5746e+05],
         [1.3637e+05, 1.3638e+05, 1.3638e+05,  ..., 1.3231e+05,
          1.3231e+05, 1.3231e+05],
         ...,
         [1.0310e+05, 1.0311e+05, 1.0312e+05,  ..., 1.0274e+05,
          1.027

In [21]:
print(label)

tensor([[[2.0130e+05, 2.0128e+05, 2.0127e+05,  ..., 1.9970e+05,
          1.9969e+05, 1.9969e+05],
         [1.6036e+05, 1.6036e+05, 1.6036e+05,  ..., 1.5738e+05,
          1.5740e+05, 1.5741e+05],
         [1.3633e+05, 1.3634e+05, 1.3634e+05,  ..., 1.3227e+05,
          1.3227e+05, 1.3226e+05],
         ...,
         [1.0308e+05, 1.0309e+05, 1.0310e+05,  ..., 1.0268e+05,
          1.0268e+05, 1.0269e+05],
         [1.9595e-01, 1.4695e-01, 1.1039e-01,  ..., 7.8946e-01,
          8.8265e-01, 9.0048e-01],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.8327e-04,
          2.0125e-04, 1.4473e-04]],

        [[2.0138e+05, 2.0138e+05, 2.0137e+05,  ..., 1.9979e+05,
          1.9979e+05, 1.9978e+05],
         [1.6041e+05, 1.6041e+05, 1.6041e+05,  ..., 1.5743e+05,
          1.5745e+05, 1.5746e+05],
         [1.3637e+05, 1.3638e+05, 1.3638e+05,  ..., 1.3231e+05,
          1.3231e+05, 1.3231e+05],
         ...,
         [1.0310e+05, 1.0311e+05, 1.0312e+05,  ..., 1.0274e+05,
          1.027